In [ ]:
%matplotlib inline

# N170: a visual index of facial recognition

Using this notebook, we will discuss and learn about the N170, an event-related potential that is associated with the perception of faces. 

In [ ]:
# Imports
import os, numpy as np,pandas as pd
import warnings
warnings.filterwarnings('ignore')
import eegnb
from eegnb import generate_save_fn
from eegnb.devices.eeg import EEG
from eegnb.experiments.visual_n170 import n170
from collections import OrderedDict
import seaborn as sns
from matplotlib import pyplot as plt

# MNE functions
from mne import Epochs,find_events
from mne.decoding import Vectorizer

# EEG-Notebooks functions
from eegnb.analysis.utils2 import load_data,plot_conditions, check_report
from eegnb.datasets import fetch_dataset

There are actually two ways of running experiments using this set-up. Below is one example provided to you for ease of adjusting what it is that you want to test. You can then use this template to build your own variation of an experiment!

### Experiment set-up

In [ ]:
# Here we can change the variables that we use for measuring experiments!

board_name = "muse2_bfn" # this can stay the same
experiment = "visual-N170" # this changes depending on the experiment, for the purposes of this notebook it can stay the same
subject_id = 1 # change this each time you have a new participant
session_nb = 1 # change this each time you have a new run
record_duration = 120 # this changes how long each session is

After this set-up, we will want to make sure we're using the right inputs before starting the experiment. Here's one way of looking at the inputs, based on where our data will be saved:

In [ ]:
# Start EEG device
eeg_device = EEG(device=board_name, serial_num='Muse-AF55') # make sure to replace the serial_num with 'Muse-[your device's serial digits]'

# Create save file name
save_fn = generate_save_fn(board_name, experiment, subject_id, session_nb)
print(save_fn)

If the subject and session are labeled with the right places, then that's where you'll find the data when the file is saved after the experiment runs. This is good to keep in mind for when you go find the data for analysis after.

Also, make sure you run the above cell once before every time you want to record the experiment. You'll see that the recording file name is time-stamped - if you don't run the cell again, you'll save over your old file with a new recording!

### Run experiment

Before starting an experiment, you want to make sure your data aren't too noisy. One way of doing this is to run a signal quality check:

In [ ]:
check_report(eeg_device)

To start the experiment, just run the block of code below after running the previous blocks. 

In [ ]:
n170.present(duration=record_duration, eeg=eeg_device, save_fn=save_fn)

If you are running the experiment multiple times, whether that's multiple sessions for a person or with different people, make sure that you are first running the previous cells where you change subject/session and also where the file is saved - otherwise, the files you generate will end up in the wrong folders!

### Visualize data

Let's get a quick look at the data and see what it looks like. For the record, it should look like this:

![N170](n170.jpg)

In [ ]:
eegnb_data_path = os.path.join(os.path.expanduser('~/'),'.eegnb', 'data')    

# to load data from a specific subject and session:
subject = 1
session = 2
# make sure that these numbers are correct, or it will run from the last time you used 'subject' and 'session', 
# including from above!

raw = load_data(subject,session,
                experiment='visual-N170', site='local', device_name='muse2_bfn',
                data_dir = eegnb_data_path)

We're going to do a couple of things here that we won't go into now, but will help the final visual look like what it does above. We'll talk more about this process when we do the analyses!

In [ ]:
raw.filter(1,30, method='iir')
# raw.plot_psd(fmin=1, fmax=30); # visualize power spectrum after filtering

# Create an array containing the timestamps and type of each stimulus (i.e. face or house)
events = find_events(raw)
event_id = {'House': 1, 'Face': 2}

# Create an MNE Epochs object representing all the epochs around stimulus presentation
epochs = Epochs(raw, events=events, event_id=event_id, 
                tmin=-0.1, tmax=0.8, baseline=None,
                reject={'eeg': 75e-6}, preload=True, 
                verbose=False, picks=[0,1,2,3])
print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)
epochs

We're looking for that sample drop % to be as low as possible. If yours is higher than 15%, check the raw number of trials that have come out. It could be that the original recording was a bit noisy, and that's okay.

Here's an example of what the next cell's output will look like: 

![results](n170_myplot.png)

In [ ]:
conditions = OrderedDict()
conditions['House'] = [1]
conditions['Face'] = [2]

fig, ax = plot_conditions(epochs, conditions=conditions, 
                          ci=97.5, n_boot=1000, title='',
                          channel_order=[0,1,3,2],
                          diff_waveform=(1, 2))

Use the below code to subsequently save your plot. Don't forget to change the name of the image file accordingly! Consider saving with the name of the person whose data this is, or anything else that will indicate to you whose file it is. 

In [ ]:
fig.savefig('n170_myplot.png')